In [1]:
import requests
import json
import urlparse
import logging
import pandas as pd
import sys
import datetime
from IPython.display import display
import redis
import os

In [2]:
class TLMtkData:
    def __init__(self,token):
        self.token=token
        self.root_url="https://api.wmcloud.com:443/data/v1/"
        self._initfuncmap()
    
    def _initfuncmap(self):
        self.funcmap={"getMktEqud":"api/market/getMktEqud.json"}
        
    def _getHttp(self,url,payload):
        head_auth = {"Authorization": "Bearer "+self.token}
        try:
            r=requests.get(url,params=payload,headers=head_auth,timeout=60)
        except:
            logging.error(url+" timeout")   
            return None
        return r
        #payload = {'key1': 'value1', 'key2': 'value2'}
    
    def getMktEqud(self,secID="",ticker="",tradeDate="",field=""):
        """request
        secID	string	3选1	证券内部编码，可通过交易代码在getSecID获取到。（可多值输入，最多输入50只）
        ticker	string	3选1	股票交易代码，如'000001'（可多值输入，最多输入50只）
        tradeDate	Date	3选1	输入一个日期，不输入其他请求参数，可获取到一天全部沪深股票日行情数据，输入格式“YYYYMMDD”
        beginDate	Date	否	起始日期，输入格式“YYYYMMDD”
        endDate	Date	否	截止日期，输入格式“YYYYMMDD”
        isOpen	Int32	否	股票今日是否开盘标记位：0-今日未开盘，1-今日有开盘
        field	string	是	可选参数，用","分隔,默认为空，返回全部字段，不选即为默认值。返回字段见下方
        """
        if (secID=="" and ticker=="" and tradeDate==""):
            raise ValueError("secID,ticker,tradeDate 至少填写一个")
        
        req_params={"secID":secID,"ticker":ticker,"tradeDate":tradeDate,"field":field}
        
        desturl=urlparse.urljoin(self.root_url,self.funcmap["getMktEqud"])
        
        #print u"请求地址:",desturl
        
        r=self._getHttp(desturl,req_params)
        
        while (r==None):
            time.sleep(20)
            r=self._getHttp(desturl,req_params)
        
        if (r.status_code!=200):
            logging.error("req "+r.url+r.status_code)
            return None
        
        
        jdata= r.json()
        if (jdata.get("data")==None):    
            if (jdata.get("retCode")=="-1"):
                logging.info(tradeDate+json.dumps(jdata))
            else:
                logging.error(tradeDate+json.dumps(jdata))      
            return None      
        return jdata
        """response
        secID	String	证券内部编码
        tradeDate	Date	交易日
        ticker	string	证券代码
        secShortName	string	证券简称
        exchangeCD	string	交易所代码
        preClosePrice	Double	昨收盘
        actPreClosePrice	Double	实际昨收盘
        openPrice	Double	今开盘
        highestPrice	Double	最高价
        lowestPrice	Double	最低价
        closePrice	Double	今收盘
        turnoverVol	Double	成交量
        turnoverValue	Double	成交金额
        dealAmount	Int32	成交笔数
        turnoverRate	Double	日换手率
        accumAdjFactor	Double	累积前复权因子，前复权是对历史行情进行调整，除权除息日的行情不受本次除权除息影响，不需进行调整。最新一次除权除息日至最新行情期间的价格不需要进行任何的调整，该期间前复权因子都是1。
        negMarketValue	Double	流通市值
        marketValue	Double	总市值
        PE	Double	滚动市盈率,将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange，返回字段PE
        PE1	Double	市盈率，将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange
        PB	Double	市净率，将于2016-01-01停止维护，推荐使用/api/market/getStockFactorsOneDay或/api/market/getStockFactorsDateRange，返回字段PB
        isOpen	Int32	是否开盘
        """  

In [3]:
def hasMtkDate(store,daystr):
    tname="mtkdate"
    daylinestr=datetime.datetime.strftime(datetime.datetime.strptime(daystr, "%Y%m%d"),'%Y-%m-%d')
    if ("/"+tname) in store.keys():
        fd=store.select("mtkdate",where=["tradeDate=='"+daylinestr+"'"])
        #print daylinestr,len(fd)
        #print len(fd)
        if len(fd)==0:
            return False
        else:
            return True
    else:
        return False

In [ ]:
if __name__=="__main__":
    log_format = "%(filename)s [%(asctime)s] [%(levelname)s] %(message)s"
    logging.basicConfig(format=log_format,datefmt='%Y-%m-%d %H:%M:%S %p',level=logging.DEBUG)
    
    # 定义一个Handler打印Debug及以上级别的日志到sys.stderr  
    console = logging.StreamHandler()  
    console.setLevel(logging.DEBUG)  
    # 设置日志打印格式  
    formatter = logging.Formatter('%(name)-12s: %(levelname)-8s %(message)s') 
    console.setFormatter(formatter)  
    # 将定义好的console日志handler添加到root logger  
    logging.getLogger("").addHandler(console)  
    
    #定义redis链接
    redishost=os.getenv('REDIS_PORT_6379_TCP_ADDR')
    r = redis.Redis(host=redishost,port=6379,db=0)
    
    #tk=TLMtkData(sys.argv[1])
    tk=TLMtkData("00b8a504099fb24bb62772d1318f69abd0ab4575734770349aaeaf6e3774557d")
    
    #begin=datetime.datetime.strptime(sys.argv[1], "%Y%m%d")   
    #end=datetime.datetime.strptime(sys.argv[2], "%Y%m%d")
    begin = datetime.date(2007, 1, 1)  
    end = datetime.date(2016,4,24)
    
    store = pd.HDFStore('/data/notebooks/mtkdata/store_day.h5')
    
    for i in range((end - begin).days+1):  
        day = begin + datetime.timedelta(days=i)
        daystr=datetime.datetime.strftime(day,'%Y%m%d')
        #filename="/data/notebooks/mtkdata/mtk_"+daystr+".pk"
        #if (os.path.exists(filename)==False):       
        if (hasMtkDate(store,daystr)==False):
            rsp=tk.getMktEqud(tradeDate=daystr)
            #print rsp
        
            if (rsp==None):
            #savetoRedis(r,"mtkdate:"+daystr,json.dumps(rsp))
            #print daystr," error"
            #rsp={"data":[]}  # 写入空数据
            #r.hset("dateList",daystr,0)
                print daystr," nodata"
                continue
            #else:
            #r.hset("dateList",daystr,1)

            df=pd.DataFrame(rsp["data"])
            
            
        
            #解决unicode转换
            types = df.apply(lambda x: pd.lib.infer_dtype(x.values))
            for col in types[types=='unicode'].index:
                df[col] = [c.encode('utf-8') for c in df[col].values]
            
            #print types
            
            #解决类型猜测错误问题
            for col in types[types=='integer'].index:
                if col not in ['isOpen']:
                    #print col
                    df[col] = df[col].astype(float)
            
            
            
            df.columns = [str(c) for c in df.columns]

            df=df.set_index(['tradeDate','secID'])
            
            #print df.dtypes
        
            store.append('mtkdate', df)
            #df.to_hdf('/data/notebooks/mtkdata/store_day.h5',"mtkdate",append=True)
    
        #savetoRedis(r,"mtkdate:"+daystr,json.dumps(rsp))
            print daystr," done"
        else:
            print daystr," has data"
    
    store.close()
    

root        : ERROR    20070101{"retCode": -1, "retMsg": "No Data Returned"}
root        : ERROR    20070102{"retCode": -1, "retMsg": "No Data Returned"}


20070101  nodata
20070102  nodata


root        : ERROR    20070103{"retCode": -1, "retMsg": "No Data Returned"}


20070103  nodata
20070104  done


root        : ERROR    20070106{"retCode": -1, "retMsg": "No Data Returned"}


...

20070311  nodata
20070312  done
20070313  done
20070314  done
20070315  done


root        : ERROR    20070317{"retCode": -1, "retMsg": "No Data Returned"}


20070316  done
20070317  nodata


root        : ERROR    20070318{"retCode": -1, "retMsg": "No Data Returned"}


20070318  nodata
20070319  done
20070320  done
20070321  done
20070322  done
